In [ ]:
# Install dependencies
!pip install -q transformers accelerate peft torch fastapi uvicorn pyngrok

: 

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel
from fastapi import FastAPI, HTTPException, Depends, Header
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok
import nest_asyncio
from huggingface_hub import login

In [ ]:
# Configuration
BASE_MODEL = "google/t5-small"
ADAPTER_MODEL = "your_username/t5-small-typst-lora"
HF_TOKEN = "your_huggingface_token_here"
API_TOKEN = "your_secure_bearer_token_here"  # Change this!
NGROK_AUTH_TOKEN = "your_ngrok_auth_token_here"  # Get from ngrok.com

In [ ]:
# Authenticate with Hugging Face
login(token=HF_TOKEN)

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(ADAPTER_MODEL)

In [ ]:
# Load base model
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    torch_dtype=torch.float32
)

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, ADAPTER_MODEL)
model.eval()
print("✅ Model loaded successfully")

In [ ]:
# FastAPI setup
app = FastAPI(title="T5 Typst Generator API")
security = HTTPBearer()

class GenerateRequest(BaseModel):
    input: str

class GenerateResponse(BaseModel):
    output: str

def verify_token(credentials: HTTPAuthorizationCredentials = Depends(security)):
    if credentials.credentials != API_TOKEN:
        raise HTTPException(status_code=401, detail="Invalid authentication token")
    return credentials.credentials

In [ ]:
@app.post("/generate", response_model=GenerateResponse)
async def generate(request: GenerateRequest, token: str = Depends(verify_token)):
    try:
        # Tokenize input
        inputs = tokenizer(
            request.input,
            return_tensors="pt",
            truncation=True,
            max_length=4096
        ).to(model.device)
        
        # Generate output with T5 - INCREASED max_length for full documents
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=4096,  # Increased from 512 to 2048
                num_beams=4,
                early_stopping=True
            )
        
        # Decode output
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        return GenerateResponse(output=generated_text)
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health")
async def health():
    return {"status": "healthy", "model": ADAPTER_MODEL}

In [ ]:
# Configure ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
nest_asyncio.apply()

In [ ]:
# Start ngrok tunnel
public_url = ngrok.connect(8002)
print(f"\n{'='*60}")
print(f"🚀 Public API Endpoint: {public_url}")
print(f"{'='*60}")
print(f"\nExample usage:")
print(f"curl -X POST {public_url}/generate \\")
print(f'  -H "Authorization: Bearer {API_TOKEN}" \\')
print(f'  -H "Content-Type: application/json" \\')
print(f'  -d \'{{"input": "your prompt here"}}\'\n')

In [ ]:
# Run FastAPI server
import threading
import time

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8002)  # Changed from 8003 to 8000

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

time.sleep(5)
print("✅ Server started in background thread")

: 

In [ ]:
import requests

# Use the ngrok URL from the output above
url = "https://microbic-leonida-disgustingly.ngrok-free.dev/generate"
headers = {
    "Authorization": "Bearer Qwen-model-token",
    "Content-Type": "application/json"
}

# Helper function to convert dict to string format
def format_input_dict(input_dict):
    """Convert input dictionary to the training format string"""
    return f"""Convert to IEEE-style Typst markup.
Metadata: Starting Page: {input_dict['starting_page']}, Academic Year: {input_dict['academic_year']}, Department: {input_dict['department']}, Project Title: {input_dict['project_title']}
Chapter Text: {input_dict['contents_txt']}"""

# Your input data as dictionary
input_data = {
    "starting_page": 8,
    "academic_year": "2025-26",
    "department": "B.E/Dept of CSE/BNMIT",
    "project_title": "Sleep Apnea Detection",
    "contents_txt": """CHAPTER- VI  :  Results and  Discussions

#figure(
  image("image008.jpg", width: 50%),
  caption: [Fig 6. 2  Gender  and  Age Distribution]
)

Fig 6. 2  shows the number of females and males  and   the distribution of people across different age groups. The x-axis shows age and the y-axis shows the  count.

#figure(
  image("image007.jpg", width: 50%),
  caption: [Fig 6.1 Sleep Duration Distribution  and  Exercise frequency during a  week]
)

The above count plots Fig 6.1 shows the distribution of sleep duration vs count, the bars on the graph represent the number of people who slept for a certain amount of time.  It also  shows the Exercise frequency vs count, Similar to the sleep duration plot, the bars represent how many people exercised a certain number of times.

#figure(
  image("image009.jpg", width: 50%),
  caption: [Fig 6. 3  Sleep apnea]
)

From the above plot we can infer that not many people are suffering with sleep apnea. But few people are suffering because of lower REM cycle and more number of awakenings

#figure(
  image("image010.jpg", width: 50%),
  caption: [Fig 6. 4  Count of Sleep Apnea by Gend er]
)

#figure(
  image("image011.jpg", width: 50%),
  caption: [Fig 6. 5  Data Analysis]
)

Fig 6. 5  shows the data set and the GUI representation of the analysis.

#figure(
  image("image012.jpg", width: 50%),
  caption: [Fig  6. 6   Sleep Apnea prediction]
)

The above Fig 6. 6  shows the sleep apnea prediction data along with the  the  gender based  distribution.From  the distribution, we can infer that many male consuming alcohol are  suffering.In  general many men have sleep apnea  than women."""
}

# Convert dict to string format
formatted_input = format_input_dict(input_data)

# Send as string
payload = {
    "input": formatted_input
}

print("Sending request to API...")
response = requests.post(url, headers=headers, json=payload)

print(f"Status Code: {response.status_code}")
print(f"Content-Type: {response.headers.get('Content-Type', 'N/A')}")

if response.status_code == 200:
    try:
        result = response.json()
        print("\n✅ SUCCESS!")
        print("\nGenerated Typst Output:")
        print("="*70)
        print(result["output"])
        print("="*70)
    except Exception as e:
        print(f"\n❌ JSON parsing error: {e}")
        print(f"Full response:\n{response.text}")
else:
    print(f"\n❌ Error: {response.status_code}")
    print(f"Response: {response.text[:500]}")